En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

## LATAM Data Engineer Challenge
---
Hola! gracias por leer!  
A continuación paso a detallar los pasos realizados y todo lo pensado para este desafio.

> #### Tech usadas  
> - PySpark v3.5.1  
> - Python v3.9.13

### Proceso mental
Bueno, después de revisar la estructura del dataset y revisar la documentación, pensé que lo primero que debía hacer era buscar una manera de sacar todos  
los **quotedTweets** ya que finalmente correpsonden a otro tweet, por ende mi idea era también tenerlos en la data para procesar todo. Y quité los duplicados por **id**.  

Siguiendo, y como quería usar pyspark creé una clase que instanciara a **Spark** para dejar ahí toda la configuración y cualquier método que me fuera útil.  
Caba descatar que todo el proceso lo corrí en local, por ende fui probando distintas configuraciones de **Spark**, según ciertos errores que me iban arrojando las ejecuciones de código.  

Principalmente sufrí con un par de errores del tipo: "_Py4JJavaError: An error occurred while calling {ABC}_". Esto ocurría casi todas las veces que usé UDFs,  
busqué varias soluciones pero no dí con ninguna así que opté por no usar UDFs.  


  
   
referencias
- [Instalar PySpark en Windows](https://medium.com/@dipan.saha/getting-started-with-pyspark-day-1-37e5e6fdc14b)
- [Documentación de PySpark config](https://spark.apache.org/docs/3.0.2/configuration.html)


---

Primero inicializo los nombres de archivos y las rutas, además de preguntar si es que existe el .zip para descomprimirlo en la carpeta **data**

In [1]:
import os
import cProfile
import pstats
from zipfile import ZipFile

# declaro nombres de archivo y rutas
json_file = "farmers-protest-tweets-2021-2-4.json"
parquet_file = "tweets.parquet"
data_folder_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
json_path = f"{data_folder_path}\\{json_file}"
parquet_path = f"{data_folder_path}\\{parquet_file}"

# checkea si existe la carpeta data, sino, la crea
if not os.path.exists(data_folder_path):
    os.makedirs(data_folder_path)

# checkea si existe el archivo tweets.json.zip, si no, checkea si existe el archivo json, si no, lanza excepción
if not os.path.exists(f"{data_folder_path}\\tweets.json.zip"):
    if not os.path.exists(f"{data_folder_path}\\{json_file}"):
        raise FileNotFoundError("Por favor descargue el archivo 'tweets.json.zip' y colóquelo en la carpeta 'data'")

# si no existe el archivo json, descomprime el archivo zip
if not os.path.exists(f"{data_folder_path}\\{json_file}"):
    with ZipFile(f"{data_folder_path}\\tweets.json.zip", 'r') as zObject: 
        zObject.extractall(path=f"{data_folder_path}")

Luego pregunta si es que existe el archivo parquet, sino, lo crea  

La función **json_to_parquet** toma el json principal (el de 389mb) y lo "desnestea". Esto va a buscar si existe algún elemento **quotedTweet**,  
y si existe une sus datos con el dataframe principal, esto para crear un gran dataset con todos los tweets, luego de ellos lo pasa a formato **parquet**.  

```python
def json_to_parquet(ruta_del_json: str, ruta_del_parquet: str) -> None
```

In [2]:
from helpers.helperFuncs import json_to_parquet
# si no encuentra el archivo parquet, lo crea
if not os.path.exists(f"{parquet_path}"):
    json_to_parquet(json_path, parquet_path)

---  

#### Pregunta 1
Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
Returns: 
[(datetime.date(1999, 11, 15), "LATAM321"), (datetime.date(1999, 7, 15), "LATAM_CHI"), ...]
```

In [3]:
%%script false --no-raise-error
from q1_memory import q1_memory
q1_mem_result = q1_memory(f"{parquet_path}")
q1_mem_result

Couldn't find program: 'false'


In [4]:
%%script false --no-raise-error
from q1_time import q1_time

with cProfile.Profile() as pr:
    q1_time_result = q1_time(f"{parquet_path}")

q1_stats = pstats.Stats(pr).strip_dirs().sort_stats('time')
q1_stats.dump_stats('./stats/q1_time')
q1_stats.print_stats()
q1_time_result

Couldn't find program: 'false'


#### Pregunta 2
Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:
```python
def q2_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns: 
[("✈️", 6856), ("❤️", 5876), ...]
```

In [5]:
%%script false --no-raise-error
from q2_memory import q2_memory
q2_mem_result = q2_memory(f"{parquet_path}")
q2_mem_result

Couldn't find program: 'false'


In [6]:
from q2_time import q2_time

with cProfile.Profile() as pr:
    q2_time_result = q2_time(f"{parquet_path}")

q2_stats = pstats.Stats(pr).strip_dirs().sort_stats('time')
q2_stats.dump_stats('./stats/q2_time')
q2_stats.print_stats()
q2_time_result

         49905 function calls (49480 primitive calls) in 60.144 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      340   56.623    0.167   56.623    0.167 {method 'recv_into' of '_socket.socket' objects}
       27    2.965    0.110    2.965    0.110 {built-in method time.sleep}
       12    0.462    0.038    0.462    0.038 {method 'acquire' of '_thread.lock' objects}
      333    0.013    0.000    0.013    0.000 {method 'sendall' of '_socket.socket' objects}
        1    0.006    0.006    0.006    0.006 {built-in method _winapi.CreateProcess}
       20    0.006    0.000    0.006    0.000 {built-in method io.open}
       18    0.005    0.000    0.005    0.000 {method 'readlines' of '_io._IOBase' objects}
       61    0.005    0.000    0.005    0.000 {built-in method nt.stat}
     5034    0.004    0.000    0.004    0.000 {method 'translate' of 'str' objects}
      333    0.004    0.000   56.640    0.170 clientserver.py:499

[('🙏', 5538),
 ('😂', 3712),
 ('🚜', 3017),
 ('🌾', 2288),
 ('🇮🇳', 2231),
 ('🤣', 1997),
 ('✊', 1776),
 ('🙏🏻', 1495),
 ('❤️', 1478),
 ('👇', 1118)]

#### Pregunta 3
El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. Debe incluir las siguientes funciones:
```python
def q3_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns: 
[("LATAM321", 387), ("LATAM_CHI", 129), ...]
```

In [7]:
%%script false --no-raise-error
from q3_memory import q3_memory
q3_mem_result = q3_memory(f"{parquet_path}")
q3_mem_result

Couldn't find program: 'false'


In [8]:
%%script false --no-raise-error
from q3_time import q3_time

with cProfile.Profile() as pr:
    q3_time_result = q3_time(f"{parquet_path}")

q3_stats = pstats.Stats(pr).strip_dirs().sort_stats('time')
q3_stats.dump_stats('./stats/q3_time')
q3_stats.print_stats()
q3_time_result

Couldn't find program: 'false'
